In [ ]:
import numpy as np
import pandas as pd
from multiprocessing import Pool
from glob import glob 
import os
from datetime import datetime,date
from dateutil.relativedelta import relativedelta as rd
import cufflinks as cf
cf.go_offline()

In [ ]:
def make_pool(func, params, threads):
    """
    Ejecuta de forma simultánea múltiples llamadas a una función
    :param func: function, objeto función a paralelizar
    :param params: iterable, parámetros de evaluación paralela
    :param threads: int, número de hilos de multiprocesamiento
    :return: list, resultado de la ejecución paralela agrupada en una lista
    """
    pool = Pool(threads)
    data = pool.starmap(func, params)
    pool.close()
    pool.join()
    del pool
    return [x for x in data]

In [ ]:
def f(x,y):
    return (x*y)**2

In [ ]:
X = list( np.arange(-10,10,0.001))
Y = list ( np.arange(-10,10,0.001))
len(X)*len(Y)

In [ ]:
l = [f(x,y) for x in X for y in Y]

In [ ]:
del l

In [ ]:
l = make_pool(f,[(x,y) for x in X for y in X],16)

In [ ]:
ruta = '/media/jose/090f6b94-de30-4aaf-9f8a-4e18b120d7f6/bd/02.  Para Ingeniería/aire/csv/*.csv'

In [ ]:
archivos = glob(ruta)
len(archivos)

In [ ]:
arch = archivos[23]

In [ ]:
def leerDatos(arch:str)->pd.DataFrame:
    data = pd.read_csv(arch)
    data['ts'] = pd.to_datetime(data['FECHA']+" "+(data['HORA'].map(int)-1).map(lambda x:f'{x:02d}:00:00'))
    data = data.drop(['FECHA','HORA'],axis=1)
    return data

In [ ]:
data = pd.concat(map(leerDatos,archivos),ignore_index=True)
data.shape

In [ ]:
data.to_parquet('aire.parquet')

In [ ]:
data  = pd.read_parquet('aire.parquet')

In [ ]:
data.head()

In [ ]:
vobs = 24 
vpred = 1
sub = 3
ini ,fin =data['ts'].min(),data['ts'].max()
print(ini,fin)
anclai = ini+rd(hours=vobs-1)
anclaf = fin-rd(hours=vpred)
anclai,anclaf
anclas = [anclai+rd(hours=i) for i in range((anclaf-anclai).days*24)]
len(anclas)

In [ ]:
def ingX(data:pd.DataFrame,ancla:datetime,sub:int)->pd.DataFrame:
    ini = ancla-rd(hours=sub-1)
    fin =  ancla
    aux = data[(data['ts']>=ini)& (data['ts']<=fin)].copy()
    funciones = [np.min,np.max,np.mean,np.std,np.median]
    piv = aux.pivot_table(index='ESTACION',columns='CONTAMINANTE',values='VALOR',aggfunc=funciones)
    piv.columns = [f'x_{x}_{y}_ult_{sub}' for x,y in piv.columns]
    piv.reset_index(inplace=True)
    piv.insert(1,'ancla',ancla)
    return piv

In [ ]:
ingX(data,datetime(2017,8,20,14),24)